In [1]:
import networkx as nx
import pylab as plt
import pandas as pd
import numpy as np
import math
from operator import itemgetter
# Create blank graph


C:\Users\allen\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:

def scaleValues(E_matrix):
    #calcolo il total weight sul grafo
    W = 0
    for i in E_matrix.columns.values:
        W += E_matrix.loc[i].sum()
    print(f'Total weight: {W}')
    
    def scale(x):
        return x/W
    
    E_matrix = E_matrix.applymap(scale)
    
    return E_matrix


def computeQ(E_matrix):
    Q = 0

    #calcolo Q iniziale 
    for i in E_matrix.columns.values:
        a_i_square = pow(E_matrix.loc[i].values.sum(),2)
        e_ii = E_matrix.loc[i][i]
        Q += e_ii - a_i_square
    
    return Q


def removeDuplicatePairs(pairs_original, matrix):

    #print(f'ORIGINAL : {pairs_original}')

    pairs_copy = pairs_original.copy()

    for i in range(len(pairs_copy)):
        pairs_copy[i] = tuple(sorted(pairs_copy[i]))

    pairs_copy = sorted(pairs_copy)
    #print(f'SORTED: {pairs_copy}')


    duplicates = []

    for i in range(len(pairs_copy)-1):
        if pairs_copy[i] == pairs_copy[i+1]:
            duplicates.append(pairs_copy[i])

    #print(f'DUPLICATES: {duplicates}')


    for i in duplicates:
        pairs_original.remove(i)
        pairs_original.remove(tuple(sorted(i, reverse = True)))


    #print(f'ORIGINAL W/O DUPLICATES : {pairs_original}')


    for i in duplicates:
        if(matrix.loc[i[0]][i[1]] > matrix.loc[i[1]][i[0]]):
            pairs_original.append(i)
        else:
            pairs_original.append(tuple(sorted(i, reverse = True)))

    #print(f'ORIGINAL UPDATED : {pairs_original}')

    return pairs_original
    



def calcolaMerge(E_matrix):   
    
    print(f'Shape starting: {E_matrix.shape}')
    
    #print(f'E_matrix scalata:\n {E_matrix}')
    

    #singola iterazione per calcolare join tra communities
    
    
    #calcolo Q iniziale
    Q = computeQ(E_matrix)
    print(f'Q iniziale: {Q}')

    #definisco df in cui memorizzare DQs
    df = pd.DataFrame(columns=['pair','DQ','newQ'])
    pairs_to_try = []

    #individuo le coppie da testare => coppie con almeno un arco tra loro
    for i in E_matrix.columns.values:

        temp = pd.DataFrame(E_matrix.loc[i])    
        possible_dest = temp[temp[i]>0].index.values

        for j in possible_dest:
            if(i!=j):
                pairs_to_try.append((i,j))
    #print(pairs_to_try)
    
    if(len(pairs_to_try)>0):
        
        pairs_to_try = removeDuplicatePairs(pairs_to_try, E_matrix)

        #calcolo DQ per ogni coppia individuata
        for k in pairs_to_try:
            i = k[0]
            j = k[1]

            a_i = E_matrix.loc[i].values.sum()
            a_j = E_matrix.loc[j].values.sum()

            DQ = 2 * (E_matrix.loc[i][j] - (a_i*a_j))
            #delta_Qs.append(DQ)

            last = len(df)
            df.loc[last,'pair'] = k #aggiungo valori
            df.loc[last,'DQ'] = DQ #aggiungo valori
            df.loc[last,'newQ'] = Q +DQ #aggiungo valori


        print(f'DQ ottenute dai merge:\n {df}')

        #identifico la coppia che fornisce maggior incremento positivo in Q => Q + DQ maggiore
        winning_pair = df.query('newQ == newQ.max()')['pair'].values[0]

        print(f'Miglior merge: {winning_pair}')


        #genero le nuove colonne merged MA NON LE AGGIUNGO SUBITO
        old_columns = E_matrix.columns.values

        #print(f'old_columns: {old_columns}')


        old_columns = np.delete(old_columns, np.argwhere(old_columns == winning_pair[0]))
        old_columns = np.delete(old_columns, np.argwhere(old_columns == winning_pair[1]))


        new_columns = np.append(old_columns,winning_pair[0]+'-'+winning_pair[1])
        #print(f'new_columns: {new_columns}')

        #genero le nuove rows merged



        #rows to merge
        merged_row = E_matrix.loc[winning_pair[0]] + E_matrix.loc[winning_pair[1]]

        #remove rows of single merged communities
        E_matrix.drop(index = winning_pair[0], inplace=True)
        E_matrix.drop(index = winning_pair[1], inplace=True)

        #add new merged row to matrix
        E_matrix.loc[winning_pair[0]+'-'+winning_pair[1]] = merged_row

        #NOW CHANGE IN THE COLUMNS

        #first compute the list of values to be insterted in the new column
        new_values = []

        #for all rows compute the merged sum value 
        for i in new_columns:
            new_value = E_matrix.loc[i][[winning_pair[0],winning_pair[1]]].sum()
            new_values.append(new_value)

        #remove the single columns of merged comm
        E_matrix = E_matrix.drop([winning_pair[0], winning_pair[1]], axis=1)

        #add the new merged column
        #E_matrix = E_matrix.insert(winning_pair[0]+winning_pair[1], new_values)


        E_matrix[winning_pair[0]+'-'+winning_pair[1]] = new_values

        #print(f'AM finale: \n {E_matrix}')
        print(f'Shape obtained: {E_matrix.shape}')

        Q_finale = computeQ(E_matrix)
        
    else:
        Q_finale = float("NaN")
    
    return E_matrix, [E_matrix.columns.values, Q_finale]

In [137]:
#definisco df con tante colonne quanti autori
E_matrix = pd.DataFrame(columns=['a','b','c'])


E_matrix.loc['a',['a','b','c']] = [0,1,2]
E_matrix.loc['b',['a','b','c']] = [2,0,3]
E_matrix.loc['c',['a','b','c']] = [1,1,0]

E_matrix

,a,b,c
a,0,1,2
b,2,0,3
c,1,1,0


In [138]:
E_matrix = scaleValues(E_matrix)

Total weight: 10


In [139]:
E_matrix2, clusters = calcolaMerge(E_matrix)
print(E_matrix2)
print(clusters)

Shape starting: (3, 3)
Q iniziale: -0.38
ORIGINAL : [('a', 'b'), ('a', 'c'), ('b', 'a'), ('b', 'c'), ('c', 'a'), ('c', 'b')]
ORIGINAL UPDATED : [('b', 'a'), ('a', 'c'), ('b', 'c')]
DQ ottenute dai merge:
      pair    DQ  newQ
0  (b, a)   0.1 -0.28
1  (a, c)  0.28  -0.1
2  (b, c)   0.4  0.02
Miglior merge: ('b', 'c')
Shape obtained: (2, 2)
       a  b-c
a    0.0  0.3
b-c  0.3  0.4
[array(['a', 'b-c'], dtype=object), -0.1800000000000001]


In [140]:
E_matrix3, clusters = calcolaMerge(E_matrix2)
print(E_matrix3)
print(clusters[-1])

Shape starting: (2, 2)
Q iniziale: -0.1800000000000001
ORIGINAL : [('a', 'b-c'), ('b-c', 'a')]
ORIGINAL UPDATED : [('b-c', 'a')]
DQ ottenute dai merge:
        pair    DQ newQ
0  (b-c, a)  0.18 -0.0
Miglior merge: ('b-c', 'a')
Shape obtained: (1, 1)
       b-c-a
b-c-a    1.0
0.0


In [6]:
DF = pd.read_csv('./AM_Matrix.csv')
DF.rename(columns={'Unnamed: 0':'From'}, inplace = True)
DF.set_index('From',inplace = True,drop=True)

for i in DF.columns.values:
    DF.loc[i][i] = 0


DF

,00168ef7c9a4fed70901c32e2a3311a60893e328859fcd615100cf40da97344e,00a5614eecf54e2d713de00b7a6a75814ca14d5732779117f77346fa01ddca12,00ef89995ea4bb4aa275935f61e7a0e088efbc28666464b11d6dd409616db1a2,00f1809b78bdb4854c678a8900fca8c0454b78bc5a41811c40f97a0ab61ae5a0,010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f653f755ec24802635e3a,012339d7d671c04790fe138a9a3b12cf8b086871a5d3f30a3e7bc8397dc90f12,018ebf6145dd4fd0a4d65369c1eb0a2c349c0c46036f46d5791fce5d7c49d08a,01ae695ad5641c0f2b920d0066852d6fe8064acd3fe7590f6230bafcd6ff534a,023e3d30c4715150b21baa4a976d36f4647f21d15f17b9703231c7d51dafedb0,02b06f1e5caa2d6b8489ffa40bb321c7b62cdf2bce0d6ef61cc38bffa2a4f8cc,...,fdd022bf453a8d620dbf0c05b7a01c1233ec15454484aa3c2dce65e13489ef9d,fde67c167fea57ba49205b24abc47bafc96f76d318e19e9183314120307dc4b6,fde7b9779edadb0549355d89d9ec330250f09f965b059ef1317659c1255ae3d8,fdfa6a73ea5fbfcbd5c7650951ec26f5fbea28d8d4b275b929673a360947b7d7,fe134f3e076ab28cdb553a309ef1940a3e3c5d454ec26b28793d1e9eb042dad5,fe8853528bdee7911b91829edffd3479ee6376168c8790665ec59f1ecde8b05c,feca281ea68a9c47ec6fb59ec5d9f31d76105ad9d058a0d20dd37f5b97d848b1,ff24acad38556b1c8d521f449c0399008632be8d96b0c25a3da0766658502646,ff665fdeae13e20a855549e1e389b3992088650b2feea22b4dbfbe0c051d5ccc,ff9af28010b1a78e3af698c64734cecf907988b927435d9a887a4cdcce248611
From,,,,,,,,,,,,,,,,,,,,,
00168ef7c9a4fed70901c32e2a3311a60893e328859fcd615100cf40da97344e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00a5614eecf54e2d713de00b7a6a75814ca14d5732779117f77346fa01ddca12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00ef89995ea4bb4aa275935f61e7a0e088efbc28666464b11d6dd409616db1a2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00f1809b78bdb4854c678a8900fca8c0454b78bc5a41811c40f97a0ab61ae5a0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f653f755ec24802635e3a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe8853528bdee7911b91829edffd3479ee6376168c8790665ec59f1ecde8b05c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
feca281ea68a9c47ec6fb59ec5d9f31d76105ad9d058a0d20dd37f5b97d848b1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff24acad38556b1c8d521f449c0399008632be8d96b0c25a3da0766658502646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
DF = scaleValues(DF)
clusters_iterations = []
while(DF.shape[0]>1):
    DF, clusters = calcolaMerge(DF)
    clusters_iterations.append(clusters)
    if(math.isnan(clusters[-1])):
        break
    else:
        print(f'Q obtained: {clusters[-1]}\n')
        print('\n########\n')
    


Total weight: 1882
Shape starting: (878, 878)
Q iniziale: -0.0023975669720750657
DQ ottenute dai merge:
                                                    pair        DQ      newQ
0     (00168ef7c9a4fed70901c32e2a3311a60893e328859fc...  0.001063 -0.001335
1     (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.001059 -0.001339
2     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001063 -0.001335
3     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001056 -0.001342
4     (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...   0.00106 -0.001338
...                                                 ...       ...       ...
1808  (d74d15bb159e8729ac988fe8db16d28f397f137cd29e3...  0.001059 -0.001338
1809  (90a41f2da7704ed79cd83a1861e3adf9e9537a574cc88...  0.001055 -0.001343
1810  (b7f1a194702f504271bc7c69dc974b9944fd2aef76631...  0.002119 -0.000279
1811  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001057 -0.001341
1812  (cbcc9ec3465f367f7de79a0ae71a56218d2c128262c69...  0.

Shape obtained: (871, 871)
Q obtained: 0.008671558170079311


########

Shape starting: (871, 871)
Q iniziale: 0.008671558170079311
DQ ottenute dai merge:
                                                    pair        DQ      newQ
0     (00168ef7c9a4fed70901c32e2a3311a60893e328859fc...  0.001063  0.009734
1     (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.001059   0.00973
2     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001056  0.009727
3     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001046  0.009717
4     (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...   0.00106  0.009731
...                                                 ...       ...       ...
1776  (cd9275d14a989731762f349209159de4f515486f56796...  0.002081  0.010753
1777  (90a41f2da7704ed79cd83a1861e3adf9e9537a574cc88...  0.001055  0.009726
1778  (b7f1a194702f504271bc7c69dc974b9944fd2aef76631...  0.002119   0.01079
1779  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001057  0.009729
1780  (c

Shape obtained: (864, 864)
Q obtained: 0.017026904021656023


########

Shape starting: (864, 864)
Q iniziale: 0.017026904021656023
DQ ottenute dai merge:
                                                    pair        DQ      newQ
0     (00168ef7c9a4fed70901c32e2a3311a60893e328859fc...  0.001063   0.01809
1     (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.001046  0.018073
2     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001056  0.018083
3     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...   0.00103  0.018057
4     (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...   0.00106  0.018087
...                                                 ...       ...       ...
1765  (9f66b6f46a9adaebd9f69738418dbea1b9b3c2b739313...  0.002093   0.01912
1766  (b7f1a194702f504271bc7c69dc974b9944fd2aef76631...  0.002119  0.019146
1767  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001057  0.018084
1768  (cccdcf360fd819c26119284d841a2492a2e84b03b1eba...  0.001046  0.018073
1769  (c

Shape obtained: (857, 857)
Q obtained: 0.024985855145395544


########

Shape starting: (857, 857)
Q iniziale: 0.024985855145395544
DQ ottenute dai merge:
                                                    pair        DQ      newQ
0     (00168ef7c9a4fed70901c32e2a3311a60893e328859fc...  0.001063  0.026049
1     (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.001046  0.026032
2     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001056  0.026042
3     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...   0.00103  0.026016
4     (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...   0.00106  0.026046
...                                                 ...       ...       ...
1748  (9f66b6f46a9adaebd9f69738418dbea1b9b3c2b739313...  0.002093  0.027079
1749  (b7f1a194702f504271bc7c69dc974b9944fd2aef76631...  0.002119  0.027104
1750  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001057  0.026043
1751  (cccdcf360fd819c26119284d841a2492a2e84b03b1eba...  0.001046  0.026032
1752  (c

Shape obtained: (850, 850)
Q obtained: 0.03399169490932046


########

Shape starting: (850, 850)
Q iniziale: 0.03399169490932046
DQ ottenute dai merge:
                                                    pair        DQ      newQ
0     (00168ef7c9a4fed70901c32e2a3311a60893e328859fc...  0.001063  0.035054
1     (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.001046  0.035038
2     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001056  0.035048
3     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...   0.00103  0.035022
4     (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...   0.00106  0.035052
...                                                 ...       ...       ...
1738  (9f66b6f46a9adaebd9f69738418dbea1b9b3c2b739313...  0.002093  0.036084
1739  (b7f1a194702f504271bc7c69dc974b9944fd2aef76631...  0.002119   0.03611
1740  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001057  0.035049
1741  (cccdcf360fd819c26119284d841a2492a2e84b03b1eba...  0.001046  0.035038
1742  (cbc

...

Shape obtained: (536, 536)
Q obtained: 0.34748458747279704


########

Shape starting: (536, 536)
Q iniziale: 0.34748458747279704
DQ ottenute dai merge:
                                                   pair        DQ      newQ
0    (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.000705   0.34819
1    (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...  0.000985  0.348469
2    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...   0.00106  0.348544
3    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...   0.00106  0.348544
4    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...  0.000673  0.348158
..                                                 ...       ...       ...
817  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56...  0.000705   0.34819
818  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56... -0.000367  0.347118
819  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56...  0.000705   0.34819
820  (f7a63d0b6ff6fb96e1d3e2edf74e14ef6cdfb143beca8...  0.000705   0.34819
821  (f9a81e05c600b90

<ipython-input-5-c40cf3812dc6>:173: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  E_matrix[winning_pair[0]+'-'+winning_pair[1]] = new_values


Shape obtained: (535, 535)
Q obtained: 0.34801593710085227


########

Shape starting: (535, 535)
Q iniziale: 0.34801593710085227
DQ ottenute dai merge:
                                                   pair        DQ      newQ
0    (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.000705  0.348721
1    (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...  0.000985  0.349001
2    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...   0.00106  0.349076
3    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...   0.00106  0.349076
4    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...  0.000673  0.348689
..                                                 ...       ...       ...
816  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56...  0.000705  0.348721
817  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56... -0.000367  0.347649
818  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56...  0.000705  0.348721
819  (f7a63d0b6ff6fb96e1d3e2edf74e14ef6cdfb143beca8...  0.000705  0.348721
820  (f9a81e05c600b90

<ipython-input-5-c40cf3812dc6>:173: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  E_matrix[winning_pair[0]+'-'+winning_pair[1]] = new_values


Shape obtained: (534, 534)
Q obtained: 0.34854728672890756


########

Shape starting: (534, 534)
Q iniziale: 0.34854728672890756
DQ ottenute dai merge:
                                                   pair        DQ      newQ
0    (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.000705  0.349253
1    (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...  0.000985  0.349532
2    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...   0.00106  0.349607
3    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...   0.00106  0.349607
4    (010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f6...  0.000673   0.34922
..                                                 ...       ...       ...
814  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56...  0.000705  0.349253
815  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56... -0.000367   0.34818
816  (d02ad1a191fea29478a1c5d9edd1da8c48e20d0d3fd56...  0.000705  0.349253
817  (f7a63d0b6ff6fb96e1d3e2edf74e14ef6cdfb143beca8...  0.000705  0.349253
818  (f9a81e05c600b90